##Script contenant les principales fonctions du projet : 

In [1]:
#Appel du script permettant de nettoyer lic_comm
#%run 0-nettoyage-bdd-v2.ipynb
#Définition du chemin d'accès des bases licenciés et population :

#chemin_lic_comm = "https://www.data.gouv.fr/fr/datasets/r/9b002e32-0ec9-40fc-9292-cddeae4416c6"
#chemmin_pop_dep = 'C:/Users/Cosima/Documents/Ensae/Projet python/pop_dep_2019.xlsx'

#lic_comm = nettoyer_lic_comm(chemin_acces_lic_comm=chemin_lic_comm, chemin_acces_pop_dep=chemmin_pop_dep)

In [2]:
#### Fonction qui donne la federation comptant le plus de licencies par âge et sexe, dans chaque région

def calculer_licencies_categ_reg(age, s) :

    if age != "Ensemble" :

        lic_reg_categ = lic_comm[(lic_comm['sexe'] == s) & (lic_comm['tranche_age'] == age)] 
        lic_reg_categ = lic_reg_categ[['Code', 'Fédération', 'Région', 'nb_licencies']].groupby(['Code', 'Fédération', 'Région']).agg(total_licencies=('nb_licencies', 'sum')).reset_index()

    
    else :

        lic_reg_categ = lic_comm[lic_comm['sexe'] == s]
        lic_reg_categ = lic_reg_categ[['Code', 'Fédération', 'Région', 'tranche_age', 'nb_licencies']].groupby(['Code', 'tranche_age', 'Fédération', 'Région']).agg(total_licencies=('nb_licencies', 'sum')).reset_index()

                                 

#On retire les fdérations qui ne précisent pas le sport pratiqué (UNSS, )
    lic_reg_categ = lic_reg_categ[(lic_reg_categ["Fédération"]!= "FF Sports Pour Tous") &
                                  (lic_reg_categ["Fédération"]!= "F Sportive Educative de l'Enseignement Catholique (UGSEL)") &
                                  (lic_reg_categ["Fédération"]!= "Union Française des Œuvres Laïques d'Éducation Physique (UFOLEP)") &
                                  (lic_reg_categ["Fédération"]!= "Union Nationale du Sport Scolaire (UNSS)") &
                                  (lic_reg_categ["Fédération"]!= "Union Sportive de l'Enseignement du Premier Degré") &
                                  (lic_reg_categ["Fédération"]!= "F Sportive et Gymnique du Travail") &
                                  (lic_reg_categ["Fédération"]!= "FF de la Retraite Sportive")]
    
    
#Selectionner la fédé ayant le max de licencies pour chaque région
    for reg in (lic_reg_categ['Région'].unique()) :
        if reg == "Auvergne-Rhône-Alpes" :   #initialisation de la table
            region = lic_reg_categ[lic_reg_categ['Région'] == reg]
            max = region['total_licencies'].max() #on stocke le nb maximal de licenciées 
            max_lic_reg_categ = region[region['total_licencies'] == max] #Selection de la ligne pour lequel le nb de licenciées est maximal : ca donne la fédé de la région qui a le plus de licenciées
        else:
            region = lic_reg_categ[lic_reg_categ['Région'] == reg]
            max = region['total_licencies'].max() #on stocke le nb maximal de licenciés 
            max_lic_reg = region[region['total_licencies'] == max] #Selection de la ligne pour lequel le nb de licenciés est maximal : ca donne la fédé de la région qui a le plus de licenciés
            max_lic_reg_categ = pd.concat([max_lic_reg_categ, max_lic_reg], ignore_index=True)

    #On stocke la fede ayant le plus de licenciés dans le finistère pour la note de lectuer
    bret = max_lic_reg_categ[max_lic_reg_categ['Région'] == "Bretagne"]
    sportbr = bret['Fédération'].unique()[0]
    
#Pour le titre et l'annotation du graphique
    if s == 'F' and age != 'Ensemble':
       titre = "Fédération comptant le plus grand nombre de licenciées par région"
       sous_titre = "{} ans".format(age)
       note = "Lecture : En 2018, la fédération comptant le plus de licenciées parmi les {} ans en Bretagne est la {}.".format(age, sportbr)

    elif s == 'H' and age != "Ensemble":
       titre = "Fédération comptant le plus grand nombre de licenciés par région"
       sous_titre = "{} ans".format(age)
       note = "Lecture : En 2018, la fédération comptant le plus de licenciés parmi les {} ans en Bretagne est la {}.".format(age, sportbr)

    elif s == 'F' and age == 'Ensemble' :
        titre = 'Fédération comptant le plus grand nombre de licenciées par région tous âges confondus'
        note = "Lecture : En 2018, la fédération comptant le plus de licenciées en Bretagne est la {}.".format(sportbr)

    elif s == 'H' and age == 'Ensemble' :
        titre = 'Fédération comptant le plus grand nombre de licenciés par région tous âges confondus'
        note = "Lecture : En 2018, la fédération comptant le plus de licenciés en Bretagne est la {}.".format(sportbr)


#Pour la carte : ajouter les codes région manuellement  

    max_lic_reg_categ['code_reg'] = max_lic_reg_categ['Région'].map({
          'Île-de-France' : '11',
          'Centre-Val de Loire' : '24',
          'Bourgogne-Franche-Comté' : '27',
          'Normandie' : '28',
          'Hauts-de-France' : '32',
          'Grand Est' : '44',
          'Pays de la Loire' : '52',
          'Bretagne' : '53',
          'Nouvelle-Aquitaine' : '75',
          'Occitanie' : '76',
          'Auvergne-Rhône-Alpes' : '84',
          "Provence-Alpes-Côte d'Azur" : "93",
          'Corse' : '94'
      })
    
    #On renomme les régions Nouvelle Acquitaine et Grand Est dans la base max_lic_reg_categ pour que les libellés correspondent
    #a ceux de la base du perimetre des régions
    max_lic_reg_categ.loc[max_lic_reg_categ['Région'] == "Grand Est", 'Région'] = "Grand-Est"
    max_lic_reg_categ.loc[max_lic_reg_categ['Région'] == "Nouvelle-Aquitaine", 'Région'] = "Nouvelle Aquitaine"


    fig = px.choropleth_mapbox(
        max_lic_reg_categ,
        geojson=contours_regions,
        locations='Région',
        featureidkey="properties.libgeo",
        color='Fédération',
        color_continuous_scale="Viridis",
        mapbox_style="carto-positron",
        zoom=3.75, center = {"lat": 47, "lon": 3.7129},
        opacity=0.5
        )

    fig.update_layout(title=titre,
                  xaxis_title='',
                  yaxis_title='')
    


    fig.update_layout(annotations=[dict(text=note,
                                    xref="paper", yref="paper",
                                    x=-0.05, y=-0.15,
                                    showarrow=False,
                                    font=dict(size=12))])
    fig.show()
    

    return(max_lic_reg_categ)



In [3]:
### Même chose par âge et sexe : faire fonction qui donne le max de licencies par age et sexe par département
def calculer_licencies_categ(age, s) :

    lic_dep_categ = lic_comm[(lic_comm['sexe'] == s) & (lic_comm['tranche_age'] == age)] 

    if age != "Ensemble" :
        lic_dep_categ = lic_comm[(lic_comm['sexe'] == s) & (lic_comm['tranche_age'] == age)] 
        lic_dep_categ = lic_dep_categ[['Code', 'Fédération', 'Département', 'Nom du département', 'nb_licencies']].groupby(['Code', 'Fédération', 'Département', 'Nom du département',]).agg(total_licencies=('nb_licencies', 'sum')).reset_index()

    
    else :
        lic_dep_categ = lic_comm[lic_comm['sexe'] == s]
        lic_dep_categ = lic_dep_categ[['Code', 'Fédération', 'Département', 'Nom du département', 'nb_licencies']].groupby(['Code', 'Fédération', 'Département', 'Nom du département',]).agg(total_licencies=('nb_licencies', 'sum')).reset_index()


#On retire les fdérations qui ne précisent pas le sport pratiqué (UNSS, )
    lic_dep_categ = lic_dep_categ[(lic_dep_categ["Fédération"]!= "FF Sports Pour Tous") &
                                  (lic_dep_categ["Fédération"]!= "FF du Sport Universitaire") &
                                  (lic_dep_categ["Fédération"]!= "F Sportive Educative de l'Enseignement Catholique (UGSEL)") &
                                  (lic_dep_categ["Fédération"]!= "Union Française des Œuvres Laïques d'Éducation Physique (UFOLEP)") &
                                  (lic_dep_categ["Fédération"]!= "Union Nationale du Sport Scolaire (UNSS)") &
                                  (lic_dep_categ["Fédération"]!= "Union Sportive de l'Enseignement du Premier Degré") &
                                  (lic_dep_categ["Fédération"]!= "F Sportive et Gymnique du Travail") &
                                  (lic_dep_categ["Fédération"]!= "FF de la Retraite Sportive")]

#Selectionner la fédé ayant le max de licencies pour chaque région
    for dep in (lic_dep_categ['Département'].unique()) :
        if dep == '01' :   #initialisation de la table
            depart = lic_dep_categ[lic_dep_categ['Département'] == dep]
            max = depart['total_licencies'].max() #on stocke le nb maximal de licenciées 
            max_lic_dep_categ = depart[depart['total_licencies'] == max] #Selection de la ligne pour lequel le nb de licenciées est maximal : ca donne la fédé de la région qui a le plus de licenciées
        else:
            depart = lic_dep_categ[lic_dep_categ['Département'] == dep]
            max = depart['total_licencies'].max() #on stocke le nb maximal de licenciés 
            max_lic_dep = depart[depart['total_licencies'] == max] #Selection de la ligne pour lequel le nb de licenciés est maximal : ca donne la fédé de la région qui a le plus de licenciés
            max_lic_dep_categ = pd.concat([max_lic_dep_categ, max_lic_dep], ignore_index=True)

    #On stocke la fede ayant le plus de licenciés dans le finistère pour la note de lecture
    fin = max_lic_dep_categ[max_lic_dep_categ['Département'] == "29"]
    sport29 = fin['Fédération'].unique()[0]
    
#Pour le titre et l'annotation du graphique
    if s == 'F' and age != 'Ensemble':
       titre = "Fédération comptant le plus grand nombre de licenciées par département"
       sous_titre = "{} ans".format(age)
       note = "Lecture : En 2018, la fédération comptant le plus de licenciées parmi les {} ans dans le Finistère est la {}.".format(age, sport29)

    elif s == 'H' and age != "Ensemble":
       titre = "Fédération comptant le plus grand nombre de licenciés par département"
       sous_titre = "{} ans".format(age)
       note = "Lecture : En 2018, la fédération comptant le plus de licenciés parmi les {} ans dans le Finistère est la {}.".format(age, sport29)

    elif s == 'F' and age == 'Ensemble' :
        titre = 'Fédération comptant le plus grand nombre de licenciées par département tous âges confondus'
        note = "Lecture : En 2018, la fédération comptant le plus de licenciées dans le Finistère est la {}.".format(sport29)

    elif s == 'H' and age == 'Ensemble' :
        titre = 'Fédération comptant le plus grand nombre de licenciés par département tous âges confondus'
        note = "Lecture : En 2018, la fédération comptant le plus de licenciés dans le Finistère est la {}.".format(sport29)



    fig = px.choropleth_mapbox(
        max_lic_dep_categ,
        geojson=counties,
        locations='Nom du département',
        featureidkey="properties.libgeo",
        color='Fédération',
        color_continuous_scale="Viridis",
        mapbox_style="carto-positron",
        zoom=3.75, center = {"lat": 47, "lon": 3.7129},
        opacity=0.5
        )

    fig.update_layout(title=titre,
                  xaxis_title='',
                  yaxis_title='')
    


    fig.update_layout(annotations=[dict(text=note,
                                    xref="paper", yref="paper",
                                    x=-0.05, y=-0.15,
                                    showarrow=False,
                                    font=dict(size=12))])
    fig.show()
    

    return(max_lic_dep_categ)



In [4]:
#Fonction qui fournit la part de licenciés de chaque fédération du département : en sortie, pie chart avec la part des
#de chaque fédération sur l'ensemble des licenciés du département, pour un département donné (selon le code_dep fournit)

def contribution_federation_zone(code_dep, reg, lib_reg, sexe, age) :
    #Si on veut sur l'ensemble population : ajouter condition if ex if sexe == tous
    if sexe == 'Ensemble' and age == 'Ensemble':
        ### Répartition des licenciés de chaque département : sur l'ensemble des licenciés du depart, cb de % fait du foot... puis affiner par classe d'âge

        # Selection d'un département
        if reg == False :

            depart = lic_comm[lic_comm['Département'] == code_dep]
# sommer le nomre total de licenciés dans le département
            depart['total_licencies'] = depart['nb_licencies'].sum()
#grouper par fed 
            part_fed = depart[['Nom du département', 'Région', 'Département', 'Fédération', 'nb_licencies','total_licencies']].groupby(['Nom du département', 'Département', 'Région', 'Fédération', 'total_licencies']).agg(nb_licencies=('nb_licencies', 'sum')).reset_index()
            part_fed['part_licencies'] = part_fed['nb_licencies']/part_fed['total_licencies']*100

        else :
            
            depart = lic_comm[lic_comm['Région'] == lib_reg]
# sommer le nomre total de licenciés dans le département
            depart['total_licencies'] = depart['nb_licencies'].sum()
#grouper par fed 
            part_fed = depart[['Région', 'Fédération', 'nb_licencies','total_licencies']].groupby(['Région', 'Fédération', 'total_licencies']).agg(nb_licencies=('nb_licencies', 'sum')).reset_index()
            part_fed['part_licencies'] = part_fed['nb_licencies']/part_fed['total_licencies']*100

    elif sexe != 'Ensemble' and age == 'Ensemble' :
        
        if reg == False :
               # Selection d'un département
            depart = lic_comm[(lic_comm['Département'] == code_dep) & (lic_comm['sexe'] == sexe)]
        # sommer le nomre total de licenciés dans le département
            depart['total_licencies'] = depart['nb_licencies'].sum()
#grouper par fed 
            part_fed = depart[['Nom du département', 'Région', 'Département', 'Fédération', 'nb_licencies','total_licencies']].groupby(['Nom du département', 'Département', 'Région', 'Fédération', 'total_licencies']).agg(nb_licencies=('nb_licencies', 'sum')).reset_index()
            part_fed['part_licencies'] = part_fed['nb_licencies']/part_fed['total_licencies']*100
            
            nom_dep = part_fed['Nom du département'].unique()[0] #Nom du département stocké dans la variable 'nom_dep' pour le graphique 


        else :
                 # Selection d'une région
            depart = lic_comm[(lic_comm['Région'] == lib_reg) & (lic_comm['sexe'] == sexe)]
        # sommer le nomre total de licenciés dans le département
            depart['total_licencies'] = depart['nb_licencies'].sum()
#grouper par fed 
            part_fed = depart[['Région',  'Fédération', 'nb_licencies','total_licencies']].groupby(['Région', 'Fédération', 'total_licencies']).agg(nb_licencies=('nb_licencies', 'sum')).reset_index()
            part_fed['part_licencies'] = part_fed['nb_licencies']/part_fed['total_licencies']*100


    else :      

        if reg == False :

            depart = lic_comm[(lic_comm['Département'] == code_dep) & (lic_comm['sexe'] == sexe) & (lic_comm['tranche_age'] == age)]
        # sommer le nomre total de licenciés dans le département
            depart['total_licencies'] = depart['nb_licencies'].sum()
    #grouper par fed 
            part_fed = depart[['Nom du département', 'Région', 'Département', 'Fédération', 'nb_licencies','total_licencies']].groupby(['Nom du département', 'Département', 'Région', 'Fédération', 'total_licencies']).agg(nb_licencies=('nb_licencies', 'sum')).reset_index()
            part_fed['part_licencies'] = part_fed['nb_licencies']/part_fed['total_licencies']*100
            
            nom_dep = part_fed['Nom du département'].unique()[0] #Nom du département stocké dans la variable 'nom_dep' pour le graphique 


        else :         
            depart = lic_comm[(lic_comm['Région'] == lib_reg) & (lic_comm['sexe'] == sexe) & (lic_comm['tranche_age'] == age)]
        # sommer le nomre total de licenciés dans la région
            depart['total_licencies'] = depart['nb_licencies'].sum()
    #grouper par fed 
            part_fed = depart[['Région', 'Fédération', 'nb_licencies','total_licencies']].groupby(['Région', 'Fédération', 'total_licencies']).agg(nb_licencies=('nb_licencies', 'sum')).reset_index()
            part_fed['part_licencies'] = part_fed['nb_licencies']/part_fed['total_licencies']*100

    max = part_fed['part_licencies'].max() #part maximale de licenciés dans une fédération stockée dans variable max
    fede_max = part_fed[part_fed['part_licencies'] == max]
    fede_max = fede_max['Fédération'].unique()[0]


    if reg == False :
            
        nom_dep = part_fed['Nom du département'].unique()[0] #Nom du département stocké dans la variable 'nom_dep' pour le graphique 
        note = "Lecture : la {} compte {:.1f}%  des licenciés du département {}.".format(fede_max, max, nom_dep)     
        titre = 'Répartition de la pratique du sport dans le {} ({})'.format(code_dep, nom_dep)

    else :

        nom_reg = part_fed['Région'].unique()[0] #Nom du département stocké dans la variable 'nom_dep' pour le graphique 
        note = "Lecture : {:.1f}%  des sportifs de la région {} sont licenciés auprès de la {}.".format(max, nom_reg, fede_max)  
        titre = 'Répartition de la pratique du sport dans la région {}'.format(nom_reg)

    #Faire un pie chart 

    #Agréger les fédérations pour lesquelles il y a moins de 1% des licenciés (categorie 'autre') sinon graph illisible

    part_fed['Fédération'] = np.where(part_fed['part_licencies'] < 1, "Autres", part_fed['Fédération'])
    # Création du diagramme circulaire 
    fig = px.pie(part_fed, names='Fédération', values='part_licencies', title=titre)
    fig.update_layout(
    annotations=[
        dict(
            text=note, showarrow=False,x=-0.05, y=-0.15
        )
    ]
)
    fig.show()

    return(part_fed)

In [1]:
#Fonction pour déterminer les caractéristiques des licencies de chaque département (est-ce qu'ils sont plus âgés..
#A peu près la même fonction que l'autre, mais arguments supplémentaires : à l'échelle régionale si region = True, affiche le graph si graph = True
def structure_zone_categ(code_dep, region, lib_reg, graph) :

    lic_categ = lic_comm
    #lic_categ = lic_comm[(lic_comm["Fédération"]!= "FF Sports Pour Tous") &
     #                             (lic_comm["Fédération"]!= "FF du Sport Universitaire") &
      #                            (lic_comm["Fédération"]!= "F Sportive Educative de l'Enseignement Catholique (UGSEL)") &
       #                           (lic_comm["Fédération"]!= "Union Française des Œuvres Laïques d'Éducation Physique (UFOLEP)") &
        #                          (lic_comm["Fédération"]!= "Union Nationale du Sport Scolaire (UNSS)") &
         #                         (lic_comm["Fédération"]!= "Union Sportive de l'Enseignement du Premier Degré ") &
          #                        (lic_comm["Fédération"]!= "F Sportive et Gymnique du Travail") &
           #                       (lic_comm["Fédération"]!= "FF de la Retraite Sportive")]
    
### Part des liencies de chaque categorie par departement : donne la répartition des âges et sexes parmi les licencies
    if region==False and graph == True:
        depart = lic_categ[lic_categ['Code département'] == code_dep] #selection du départ

#Compter le total de licencies dans le depart
        total_lic = sum(depart['nb_licencies']) #nb total de licences du département

#Ajout du total de licences du depart dans le data frame depart
        depart['total_lic'] = total_lic
        nom_dep = depart['Nom du département'].unique()[0] #Libelle du depart stocke dans une variable pour la carte
        depart = depart.groupby(["Nom du département", "Code département", "Région", "tranche_age", "sexe", "total_lic"]).agg(part_licencies=('nb_licencies', 'sum')).reset_index()
        depart['part_licencies'] = depart['part_licencies']/depart['total_lic']*100

        max_df =depart[['part_licencies', 'tranche_age']].groupby('tranche_age').agg(part_licencies=('part_licencies', 'sum')).reset_index()
        max = max_df['part_licencies'].max() #La part maximale de licencies est stockee dans la variable max pour la note de lecture du graphique
        cat_max = max_df[max_df['part_licencies'] == max] #selection de la ligne où il y a le maximum de licencies
        cat_max = cat_max['tranche_age'].unique()[0] #la catégorie qui représente la part la plus élevée de licenciés du département est stockée dans la variable 'cat_max'

    #Diagramme
        titre = "Répartition des licenciés selon l'âge et le sexe dans le {} ({})".format(code_dep, nom_dep)
        note = "Lecture : En 2018, la tranche d'âge qui compte le plus de licenciés du département est la catégorie des {} ans, avec {:.1f}% des licenciés.".format(cat_max, max)
        fig = px.pie(depart, names='tranche_age',  values='part_licencies', hover_data= ['part_licencies', 'sexe'], title = titre)
        fig.update_layout(annotations=[dict(text=note,
                                    xref="paper", yref="paper",
                                    x=0.5, y=-0.2,
                                    showarrow=False,
                                    font=dict(size=12))])
        fig.show() 

        return(depart)
    
    elif region == False and graph == False :

        depart = lic_categ[lic_categ['Code département'] == code_dep] #selection du départ

#Compter le total de licencies dans le depart
        total_lic = sum(depart['nb_licencies']) #nb total de licences du département

#Ajout du total de licences du depart dans le data frame depart
        depart['total_lic'] = total_lic
        nom_dep = depart['Nom du département'].unique()[0] #Libelle du depart stocke dans une variable pour la carte
        depart = depart.groupby(['Nom du département', 'Code département', "Région", "tranche_age", "sexe", "total_lic"]).agg(part_licencies=('nb_licencies', 'sum')).reset_index()
        depart['part_licencies'] = depart['part_licencies']/depart['total_lic']*100

        return(depart)

    elif region == True and graph == True :
        structure_region = lic_categ[lic_categ['Région'] == lib_reg] #selection de la région

#Compter le total de licencies dans le depart
        total_lic = sum(structure_region['nb_licencies']) #nb total de licences du département

#Ajout du total de licences du depart dans le data frame depart
        structure_region['total_lic'] = total_lic
        structure_region = structure_region.groupby(["Région", "tranche_age", "sexe", "total_lic"]).agg(part_licencies=('nb_licencies', 'sum')).reset_index()
        structure_region['part_licencies'] = structure_region['part_licencies']/structure_region['total_lic']*100

        max_df =structure_region[['part_licencies', 'tranche_age']].groupby('tranche_age').agg(part_licencies=('part_licencies', 'sum')).reset_index()
        max = max_df['part_licencies'].max() #La part maximale de licencies est stockee dans la variable max pour la note de lecture du graphique
        cat_max = max_df[max_df['part_licencies'] == max] #selection de la ligne où il y a le maximum de licencies
        cat_max = cat_max['tranche_age'].unique()[0] #la catégorie qui représente la part la plus élevée de licenciés du département est stockée dans la variable 'cat_max'
        
    #Diagramme
        titre = "Répartition des licenciés selon l'âge et le sexe dans la région {}".format(lib_reg)
        note = "Lecture : En 2018, la tranche d'âge qui compte le plus de licenciés dans la région {} est la catégorie des {} ans, avec {:.1f}% des licenciés.".format(lib_reg, cat_max, max)
        fig = px.pie(structure_region, names='tranche_age',  values='part_licencies', hover_data= ['part_licencies', 'sexe'], title = titre)
        fig.update_layout(annotations=[dict(text=note,
                                    xref="paper", yref="paper",
                                    x=0.5, y=-0.2,
                                    showarrow=False,
                                    font=dict(size=12))])
        fig.show()
    
        return(structure_region)
    
    else :
        structure_region = lic_categ[lic_categ['Région'] == lib_reg] #selection de la région
#Compter le total de licencies dans le depart
        total_lic = sum(structure_region['nb_licencies']) #nb total de licences du département
#Ajout du total de licences du depart dans le data frame depart
        structure_region['total_lic'] = total_lic
        structure_region = structure_region.groupby(["Région", "tranche_age", "sexe", "total_lic"]).agg(part_licencies=('nb_licencies', 'sum')).reset_index()
        structure_region['part_licencies'] = structure_region['part_licencies']/structure_region['total_lic']*100

        return(structure_region)
    

In [7]:
#Fonction qui permet de comparer la part de licenciés d'une cartaine catégorie d'âge entre chaque région/département

#base = depart_licencies si on s'interesse aux départements
#base  = region_licencies si on s"intéresse aux régions

def graphique_age_licencies(age, region) :
     
    for reg in lic_comm['Région'].unique() :
        if reg=="Auvergne-Rhône-Alpes": #Initialisation de la table
            region_licencies = structure_zone_categ('01', region= True, lib_reg = reg, graph = False)
        else :
            b = structure_zone_categ('01', True, reg, graph = False)
            region_licencies = pd.concat([region_licencies, b], ignore_index=True)

#Même chose avec les départements mais sans le graphique (graph == False)

    for dep in lic_comm['Département'].unique() :
        if dep=="01": #Initialisation de la table
            depart_licencies = structure_zone_categ("01", region= False, lib_reg = reg, graph = False)
        else :
            c = structure_zone_categ(dep, region = False, lib_reg =reg, graph = False)
            depart_licencies = pd.concat([depart_licencies, c], ignore_index=True)
     
    if region == False :
        titre = "Proportion des licenciés de la tranche d'âge {} ans dans chaque département".format(age)
        age_zone = depart_licencies[depart_licencies['tranche_age'] == age]
        max_df = age_zone[['part_licencies', 'Nom du département', 'tranche_age']].groupby(['tranche_age', 'Nom du département']).agg(part_licencies=('part_licencies', 'sum')).reset_index()
        max = max_df['part_licencies'].max() #La part maximale de licencies est stockee dans la variable max pour la note de lecture du graphique
        dep_max = max_df[max_df['part_licencies'] == max] #selection de la ligne où il y a le maximum de licencies
        dep_max = dep_max['Nom du département'].unique()[0] #la région qui compte la part la plus élevée de licenciés de la tranche d'âge est stockée dans la variable 'cat_max'
        note = "Lecture : En 2019, le département {} compte le plus de licenciés de la catégorie des {} ans, qui représente {:.1f}% des licenciés du département.".format(dep_max, age, max)

# graphique
        fig = px.bar(age_zone, x='Nom du département', y='part_licencies', color = 'sexe')

        fig.update_traces(textposition='outside', showlegend=False) 
        fig.update_layout(title=titre,
                          xaxis_title='',
                          yaxis_title='Part de licenciés')
        fig.update_layout(annotations=[dict(text=note,
                                            xref="paper", yref="paper",
                                            x=-0.15, y=-1.75,
                                            showarrow=False,
                                            font=dict(size=10))])
        fig.show()

        return(depart_licencies)

    else :
        titre = "Proportion des licenciés de la tranche d'âge {} ans dans chaque région".format(age)
        age_zone = region_licencies[region_licencies['tranche_age'] == age]
        max_df = age_zone[['part_licencies', 'Région', 'tranche_age']].groupby(['Région', 'tranche_age']).agg(part_licencies=('part_licencies', 'sum')).reset_index()
        max = max_df['part_licencies'].max() #La part maximale de licencies est stockee dans la variable max pour la note de lecture du graphique
        reg_max = max_df[max_df['part_licencies'] == max] #selection de la ligne où il y a le maximum de licencies
        reg_max = reg_max['Région'].unique()[0] #la région qui compte la part la plus élevée de licenciés de la tranche d'âge est stockée dans la variable 'cat_max'
        note = "Lecture : En 2019, la région {} compte le plus de licenciés de la catégorie des {} ans, qui représente {:.1f}% des licenciés de la région.".format(reg_max, age, max)
        

# graphique interactif avec plotly
        fig = px.bar(age_zone, x='Région', y='part_licencies', color = 'sexe')

        fig.update_traces(textposition='outside', showlegend=False) 

        fig.update_layout(title=titre,
                          xaxis_title='',
                          yaxis_title='Part de licenciés')

        fig.update_layout(annotations=[dict(text=note,
                                            x=-0.05, y=-0.5,
                                            xref="paper", yref="paper",
                                            showarrow=False,
                                            font=dict(size=11))])
        fig.show()

        return(region_licencies)

In [ ]:
# Iterer sur les features et extraire les propriétés
#for feature in contours_regions['features']:
 #   properties = feature.get('properties', {})
  #  print(properties)